# Getting Financial Data - Pandas Datareader

### Introduction:

This time you will get data from a website.


### Step 1. Import the necessary libraries

In [6]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


### Step 2. Create your time range (start and end variables). The start date should be 01/01/2015 and the end should today (whatever your today is).

In [3]:
time_range = pd.date_range('2015-01-01', '2024-11-21')
print(time_range)

DatetimeIndex(['2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04',
               '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-10',
               ...
               '2024-11-12', '2024-11-13', '2024-11-14', '2024-11-15',
               '2024-11-16', '2024-11-17', '2024-11-18', '2024-11-19',
               '2024-11-20', '2024-11-21'],
              dtype='datetime64[ns]', length=3613, freq='D')


### Step 3. Get an API key for one of the APIs that are supported by Pandas Datareader, preferably for AlphaVantage.

If you do not have an API key for any of the supported APIs, it is easiest to get one for [AlphaVantage](https://www.alphavantage.co/support/#api-key). (Note that the API key is shown directly after the signup. You do *not* receive it via e-mail.)

(For a full list of the APIs that are supported by Pandas Datareader, [see here](https://pydata.github.io/pandas-datareader/readers/index.html). As the APIs are provided by third parties, this list may change.)

### Step 4. Use Pandas Datarader to read the daily time series for the Apple stock (ticker symbol AAPL) between 01/01/2015 and today, assign it to df_apple and print it.

In [37]:
Api_key = 'T97TPD64W11UXGW2'

df_apple = pdr.get_data_alphavantage('AAPL', api_key=Api_key)
df_apple.index = pd.to_datetime(df_apple.index)
df_apple = df_apple.loc[time_range[0]:time_range[-1]]

In [38]:
df_apple.head()

,open,high,low,close,volume
2015-01-02,111.39,111.44,107.350,109.33,53204626
2015-01-05,108.29,108.65,105.410,106.25,64285491
2015-01-06,106.54,107.43,104.630,106.26,65797116
2015-01-07,107.20,108.20,106.695,107.75,40105934
2015-01-08,109.23,112.15,108.700,111.89,59364547


### Step 5. Add a new column "stock" to the dataframe and add the ticker symbol

In [39]:
df_apple['stock'] = 'AAPL'

### Step 6. Repeat the two previous steps for a few other stocks, always creating a new dataframe: Tesla, IBM and Microsoft. (Ticker symbols TSLA, IBM and MSFT.)

In [41]:
tesla = pdr.get_data_alphavantage('TSLA', api_key=Api_key)
tesla.index = pd.to_datetime(tesla.index)
tesla['stock'] = 'TSLA'
tesla = tesla.loc[time_range[0]:time_range[-1]]

ibm = pdr.get_data_alphavantage('IBM', api_key=Api_key)
ibm.index = pd.to_datetime(ibm.index)
ibm['stock'] = 'IBM'
ibm = ibm.loc[time_range[0]:time_range[-1]]

microsfot = pdr.get_data_alphavantage('MSFT', api_key=Api_key)
microsfot.index = pd.to_datetime(microsfot.index)
microsfot['stock'] = 'MSFT'
microsfot = microsfot.loc[time_range[0]:time_range[-1]]


### Step 7. Combine the four separate dataFrames into one combined dataFrame df that holds the information for all four stocks

In [42]:
stocks = pd.concat([df_apple, tesla, ibm, microsfot])

### Step 8. Shift the stock column into the index (making it a multi-level index consisting of the ticker symbol and the date).

In [50]:
stocks = stocks.set_index(['stock', stocks.index])

### Step 7. Create a dataFrame called vol, with the volume values.

In [57]:
vol_df = stocks[['volume']]
vol_df.tail()

volume
stock                     
MSFT  2024-11-14  30246881
      2024-11-15  28247644
      2024-11-18  24742013
      2024-11-19  18133529
      2024-11-20  19191655

### Step 8. Aggregate the data of volume to weekly.
Hint: Be careful to not sum data from the same week of 2015 and other years.

In [63]:
# Garantir que o índice 'date' seja interpretado como datetime
vol_df.index = vol_df.index.set_levels(
    [vol_df.index.levels[0], pd.to_datetime(vol_df.index.levels[1])]
)

# Agrupar por 'stock' e por semana
weekly_volume = vol_df.groupby([
    pd.Grouper(level='stock'), pd.Grouper(level=1, freq='W')
]).sum()

weekly_volume.head()


volume
stock                      
AAPL  2015-01-04   53204626
      2015-01-11  282868187
      2015-01-18  304226647
      2015-01-25  198737041
      2015-02-01  465842684

### Step 9. Find all the volume traded in the year of 2015

In [66]:
vol_df = vol_df.sort_index()
vol_df_2015 = vol_df.loc[(slice(None), slice('2015-01-01', '2015-12-31')), :]

# Agrupar por 'stock' e somar os volumes para 2015
volume_2015 = vol_df_2015.groupby('stock').sum()

volume_2015


,volume
stock,
AAPL,13064316775
IBM,1105545521
MSFT,9057582311
TSLA,1086708380
